In [2]:
import pandas as pd
import numpy as np
import mplfinance as mpf
from loguru import logger

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas_datareader.data as web
import datetime
from typing import List
import plotly_express as px



In [3]:
cur_dir = 'E:/download/stock/discovery_strategy/nbs/plot_strategies'
# read csv file and plot the data
strategy_file_path = f'{cur_dir}/Strategy_Stock_Benefit_Overview.csv'
strategy_file_path = f'{cur_dir}/strategy_benefits/2ad7e829421001100035.csv'


In [4]:
def parse_data_list(buy_data_str):
    buy_data_list = []
    for buy_data in buy_data_str.split(','):
        buy_data_list.append(buy_data.split(' ')[0])
    return buy_data_list


# buy_str = "20220107 09:34:00,20220304 09:31:00,20220311 09:31:00,20221011 09:32:00"
# parsed_buy_data_list = parse_data_list(buy_str)
# print(parsed_buy_data_list)

def parse_price_data(price_str):
    """
    Parse price data from a CSV file.
    """
    pricce_data_list = []
    if type(price_str) == str:
        for buy_data in price_str.split(','):
            pricce_data_list.append(float(buy_data))
    else:
        pricce_data_list.append(float(price_str))
    return pricce_data_list

# parse_data_list("9.5,8.16,7.28")

class StockBuySellInfo:
    def __init__(self, code, buy_price, buy_date, sell_price, sell_date):
        self.code = code
        self.buy_price = buy_price
        self.buy_date = buy_date
        self.sell_price = sell_price
        self.sell_date = sell_date
        
    def __str__(self):
        return f"{self.code} - buy_price: {self.buy_price}, buy_date: {self.buy_date}, sell_price: {self.sell_price}, sell_date: {self.sell_date}"


class StrategyInfo():
    def __init__(self, sid, open, high, close, low, volume, date, inc_volume, dsc_volume):
        self.name = sid
        self.open = open
        self.high = high
        self.close = close
        self.low = low
        self.volume = volume
        self.date = date
        self.inc_volume = inc_volume
        self.dsc_volume = dsc_volume
        


def get_trade_date_list(trade_date_file: str) -> List[str]:
    # read pandas dataframe from csv file
    df = pd.read_csv(trade_date_file)
    df['cal_date'] = df['cal_date'].astype(str)
    # extract trade date list from dataframe
    trade_date_list = df['cal_date'].tolist()
    trade_date_list.sort()
    return trade_date_list


# read csv file and plot the data
stock_kline_date_file_path = f'{cur_dir}/all_stocks.csv'

def get_stock_date_kline_data():
    # key is trade_date, stock, value is stock data
    stock_kline_date_dict = {}
    stock_kline_date_data = pd.read_csv(stock_kline_date_file_path)
    logger.info("stock_kline_date_data {}", len(stock_kline_date_data))
    # group by date and symbol
    stock_kline_data_list = stock_kline_date_data.to_dict(orient='records')
    for stock_kline_data in stock_kline_data_list:
        trade_date = stock_kline_data['trade_date']
        trade_date = trade_date.replace('-', '')
        code = stock_kline_data['code']
        if trade_date not in stock_kline_date_dict:
            stock_kline_date_dict[str(trade_date)] = {}

        stock_kline_date_dict[trade_date][str(code)] = stock_kline_data
    return stock_kline_date_dict


stock_kline_date_dict = get_stock_date_kline_data()

2024-07-15 22:30:36.018 | INFO     | __main__:get_stock_date_kline_data:69 - stock_kline_date_data 5754076


In [5]:
def calculate_strategy(strategy_file_path, stock_kline_date_dict, sid="股票日K线图", mode='ALL'):
    strategy_data = pd.read_csv(strategy_file_path) 
    # strategy_data.columns
    trade_date_file = f'{cur_dir}/trade_cal.csv'
    # strategy_data column is ['StrategyID', 'SCode', 'MarketType', 'BuyDateList', 'BuyPriceList', 'SellDateList', 'SellPriceList'], merge SCode and MarketType
    strategy_data['MarketType_new'] = strategy_data['MarketType'].apply(lambda x: ['.XSHG', '.XSHE'][int(x)-1])
    strategy_data['SCode_MarketType'] = strategy_data['SCode'].astype(str) + strategy_data['MarketType_new'].astype(str)
    # strategy_data to row dict 
    rows = strategy_data.to_dict(orient='records')
    stock_buy_state = {}

    for row in rows:
        stock = row['SCode_MarketType']
        stock = stock.rjust(11, '0')
        buy_date_list = parse_data_list(row['BuyDateList'])
        sell_date_list = parse_data_list(row['SellDateList'])
        buy_price_list = parse_price_data(row['BuyPriceList'])
        sell_price_list = parse_price_data(row['SellPriceList'])
        for idx, buy_date in enumerate(buy_date_list):
            if buy_date not in stock_buy_state:
                stock_buy_state[buy_date] = {}
            stock_buy_state[buy_date][stock] = StockBuySellInfo(stock, buy_price_list[idx], buy_date, sell_price_list[idx], sell_date_list[idx])
    
        ...
    strategy_date_list = list(stock_buy_state.keys())
    strategy_date_list.sort()

    # len(strategy_date_list)
    calendar_date_list = get_trade_date_list(trade_date_file)

    # 计算每天的股票池子
    first_buy_date = strategy_date_list[0]
    # logger.info("first_buy_date: {}", first_buy_date)
    first_buy_date_index = calendar_date_list.index(first_buy_date)
    logger.info("first_buy_date_index: {}, first_buy_date {}", first_buy_date_index, first_buy_date)

    last_buy_index = calendar_date_list.index(strategy_date_list[-1])

    holding_stock_list = []
    strategy_info_list = []

    for i in range(first_buy_date_index, last_buy_index + 1):
        current_date = calendar_date_list[i]
        current_date_index = i
        # logger.info("current_date: {}", current_date)
        open_chg_pct = 0
        close_chg_pct = 0
        high_chg_pct = 0
        low_chg_pct = 0
        volume = 0
        inc_volume = 0
        dsc_volume = 0
        
        incr_stock_list = []
        clac_stock_list = []
        
        if current_date in stock_buy_state:
            # 计算当前日期的股票池子
            stock_buy_dict: dict = stock_buy_state[current_date]
            inc_volume += len(stock_buy_dict)
            incr_stock_list.extend(list(stock_buy_dict.values()))
        if current_date == '20240129':
            stock_code_set = set([stock.code for stock in holding_stock_list])
            logger.info("len holding_stock_list: {}, stock_code_set {}", len(holding_stock_list), stock_code_set)
        # logger.info("current_date {} holding_stock_list: {}", current_date, len(holding_stock_list))
        
        sell_stock_list = []
        for holding_stock in holding_stock_list:
            if holding_stock.sell_date == current_date:
                sell_stock_list.append(holding_stock)
        
        dsc_volume = len(sell_stock_list)
        for sell_stock in sell_stock_list:
            holding_stock_list.remove(sell_stock)
                
        if mode == 'ALL':
            clac_stock_list.extend(holding_stock_list)
            clac_stock_list.extend(incr_stock_list)
        elif mode == 'INCR':
            clac_stock_list.extend(incr_stock_list)
        
        # 计算当前策略的每天的收益
        for holding_stock in clac_stock_list:
            code = holding_stock.code
            if code in stock_kline_date_dict[current_date]:
                stock_info = stock_kline_date_dict[current_date][code]
                open_chg_pct += 100 * (stock_info['open']- stock_info['pre_close']) / stock_info['pre_close']
                high_chg_pct += 100 * (stock_info['high'] - stock_info['pre_close']) / stock_info['pre_close']
                close_chg_pct += 100 * (stock_info['close'] - stock_info['pre_close']) / stock_info['pre_close']
                low_chg_pct += 100 * (stock_info['low'] - stock_info['pre_close']) / stock_info['pre_close']
            else:
                ...
                # logger.info("code {} not in stock_kline_date_dict[{}]", code, current_date) 
        # 去除需要卖出的股票
        holding_stock_list.extend(incr_stock_list)
        volume = len(holding_stock_list)
        strategy_info_list.append(StrategyInfo(sid, open_chg_pct, high_chg_pct, close_chg_pct, low_chg_pct, volume, current_date, inc_volume, dsc_volume))
            
            
    # len(strategy_info_list)
    # write to csv file
    # d = {'col1': [1, 2], 'col2': [3, 4]}
    dict_list = {'open': [], 'close': [], 'high': [], 'low': [], 'volume': [], 'date': [], 'inc_volume': [], 'dsc_volume': []}
    for strategy_info in strategy_info_list:
        dict_list['open'].append(strategy_info.open)
        dict_list['close'].append(strategy_info.close)
        dict_list['high'].append(strategy_info.high)
        dict_list['low'].append(strategy_info.low)
        dict_list['volume'].append(strategy_info.volume)
        dict_list['inc_volume'].append(strategy_info.inc_volume)
        dict_list['dsc_volume'].append(strategy_info.dsc_volume)
        dict_list['date'].append(strategy_info.date)

    strategy_info_df = pd.DataFrame(dict_list)
    strategy_info_df.to_csv('strategy_info.csv', index=False)

    
    
def plot_strategy(strategy_file_path, stock_kline_date_dict, sid="股票日K线图", mode='ALL'):
    calculate_strategy(strategy_file_path, stock_kline_date_dict, sid=sid, mode=mode)
    strategy_info_data_file = f'{cur_dir}/strategy_info.csv'
    df = pd.read_csv(strategy_info_data_file)
    df['Date'] = pd.to_datetime(df['date'], format='%Y%m%d')

    # fig = make_subplots(rows=4, cols=1, shared_xaxes=True,
    #                     vertical_spacing=0.05, subplot_titles=('日K', 'ALL', 'Buy', 'SELL'),
    #                     row_width=[ 0.1, 0.1, 0.2, 0.7])
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    fig.add_trace(go.Candlestick(
        x=df['Date'],
        open=df['open'] * 10,
        high=df['high'] * 10,
        low=df['low'] * 10,
        close=df['close'] * 10,
        increasing=dict(line=dict(color='red')), # 涨的颜色
        decreasing=dict(line=dict(color='green')), # 跌的颜色
        name='日K'
    ),row=1, col=1)

    fig.add_trace(go.Bar(x=df['Date'], y=df['volume'], name='成交量'), row=2, col=1)
    # fig.add_trace(go.Bar(x=df['Date'], y=df['volume'], name='成交量'), row=3, col=1)
    # fig.add_trace(go.Bar(x=df['Date'], y=df['volume'], name='成交量'), row=4, col=1)
    # fig.add_trace(go.Bar(x=df['Date'], y=df['inc_volume'], name='买量'), row=3, col=1)
    # fig.add_trace(go.Bar(x=df['Date'], y=df['dsc_volume'], name='卖量'), row=4, col=1)
    # 设置布局
    # fig.update_layout(
    #     title=sid,
    #     xaxis_title='Date',
    #     yaxis_title='Price'
    # )
    # # 设置图表属性
    fig.update_layout(title=sid,
                    xaxis_title='',
                    yaxis_title='价格',
                    xaxis2_title='日期',
                    yaxis2_title='成交量',
                    yaxis2=dict(type='linear'))

    # 显示图表
    fig.show()
    # volume_df = df[['date', 'volume', 'inc_volume', 'dsc_volume']]
    # dict_list = {'date': [], 'volume_type': [], 'volume': []}
    
    # for row in volume_df.to_dict(orient='records'):
    #     date = row['date']
    #     dict_list['date'].append(date)
    #     dict_list['volume_type'].append('ALL')
    #     dict_list['volume'].append(row['volume'])
        
    #     dict_list['date'].append(date)
    #     dict_list['volume_type'].append('BUY')
    #     dict_list['volume'].append(row['inc_volume'])
        
    #     dict_list['date'].append(date)
    #     dict_list['volume_type'].append('SELL')
    #     dict_list['volume'].append(row['dsc_volume'])
    
    # volume_df = pd.DataFrame(dict_list)
    # volume_df['Date'] = pd.to_datetime(volume_df['date'], format='%Y%m%d')
    # fig = px.bar(volume_df,
    #         x="Date",
    #         y="volume",
    #         color="volume_type")
    # fig.show()
    ...
    

def plot_strategy_v2(strategy_file_path, stock_kline_date_dict, sid="股票日K线图"):
    calculate_strategy(strategy_file_path, stock_kline_date_dict, sid=sid, mode='ALL')
    strategy_info_data_file = f'{cur_dir}/strategy_info.csv'
    df = pd.read_csv(strategy_info_data_file)
    df['Date'] = pd.to_datetime(df['date'], format='%Y%m%d')

    # fig = make_subplots(rows=4, cols=1, shared_xaxes=True,
    #                     vertical_spacing=0.05, subplot_titles=('日K', 'ALL', 'Buy', 'SELL'),
    #                     row_width=[ 0.1, 0.1, 0.2, 0.7])
    
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
    fig.add_trace(go.Candlestick(
        x=df['Date'],
        open=df['open'] * 10,
        high=df['high'] * 10,
        low=df['low'] * 10,
        close=df['close'] * 10,
        increasing=dict(line=dict(color='red')), # 涨的颜色
        decreasing=dict(line=dict(color='green')), # 跌的颜色
        name='日K'
    ),row=1, col=1)

    # fig.add_trace(go.Bar(x=df['Date'], y=df['volume'], name='成交量'), row=2, col=1)
    # fig.add_trace(go.Bar(x=df['Date'], y=df['volume'], name='成交量'), row=3, col=1)
    # fig.add_trace(go.Bar(x=df['Date'], y=df['volume'], name='成交量'), row=4, col=1)
    # fig.add_trace(go.Bar(x=df['Date'], y=df['inc_volume'], name='买量'), row=3, col=1)
    # fig.add_trace(go.Bar(x=df['Date'], y=df['dsc_volume'], name='卖量'), row=4, col=1)
    # 设置布局
    fig.update_layout(
        title='持仓',
        xaxis_title='Date',
        yaxis_title='Price'
    )
    # # 设置图表属性
    # fig.update_layout(title=sid,
    #                 xaxis_title='',
    #                 yaxis_title='价格',
    #                 xaxis2_title='日期',
    #                 yaxis2_title='成交量',
    #                 yaxis2=dict(type='linear'))

    # 显示图表
    fig.show()
    
    calculate_strategy(strategy_file_path, stock_kline_date_dict, sid=sid, mode='INCR')
    strategy_info_data_file = f'{cur_dir}/strategy_info.csv'
    df = pd.read_csv(strategy_info_data_file)
    df['Date'] = pd.to_datetime(df['date'], format='%Y%m%d')
 
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
    fig.add_trace(go.Candlestick(
        x=df['Date'],
        open=df['open'] * 10,
        high=df['high'] * 10,
        low=df['low'] * 10,
        close=df['close'] * 10,
        increasing=dict(line=dict(color='red')), # 涨的颜色
        decreasing=dict(line=dict(color='green')), # 跌的颜色
        name='日K'
    ),row=1, col=1)
 
    fig.update_layout(
        title="买入增量",
        xaxis_title='Date',
        yaxis_title='Price'
    )
    fig.show()
    
    
    volume_df = df[['date', 'volume', 'inc_volume', 'dsc_volume']]
    dict_list = {'date': [], 'volume_type': [], 'volume': []}
    
    for row in volume_df.to_dict(orient='records'):
        date = row['date']
        dict_list['date'].append(date)
        dict_list['volume_type'].append('ALL')
        dict_list['volume'].append(row['volume'])
        
        dict_list['date'].append(date)
        dict_list['volume_type'].append('BUY')
        dict_list['volume'].append(row['inc_volume'])
        
        dict_list['date'].append(date)
        dict_list['volume_type'].append('SELL')
        dict_list['volume'].append(row['dsc_volume'])
    
    volume_df = pd.DataFrame(dict_list)
    volume_df['Date'] = pd.to_datetime(volume_df['date'], format='%Y%m%d')
    fig = px.bar(volume_df,
            title="交易量",
            x="Date",
            y="volume",
            color="volume_type",
            color_discrete_map={'BUY': 'blue', 'SELL': 'green', 'ALL': 'red'})
    fig.show()

In [6]:
strategy_file_path = f'{cur_dir}/strategy_benefits/2ad7e829421001100035.csv'
# plot_strategy_v2(strategy_file_path, stock_kline_date_dict)
plot_strategy(strategy_file_path, stock_kline_date_dict, mode='ALL')

2024-07-15 22:31:07.740 | INFO     | __main__:calculate_strategy:35 - first_buy_date_index: 484, first_buy_date 20240102
2024-07-15 22:31:07.802 | INFO     | __main__:calculate_strategy:64 - len holding_stock_list: 1871, stock_code_set {'603848.XSHG', '300856.XSHE', '600758.XSHG', '301030.XSHE', '000932.XSHE', '688153.XSHG', '688707.XSHG', '002023.XSHE', '300690.XSHE', '600064.XSHG', '600172.XSHG', '000910.XSHE', '002478.XSHE', '002697.XSHE', '601788.XSHG', '001696.XSHE', '605339.XSHG', '603311.XSHG', '600590.XSHG', '603677.XSHG', '300945.XSHE', '002446.XSHE', '600573.XSHG', '603355.XSHG', '301227.XSHE', '300498.XSHE', '002727.XSHE', '002949.XSHE', '300654.XSHE', '301238.XSHE', '301046.XSHE', '600300.XSHG', '002609.XSHE', '688055.XSHG', '002658.XSHE', '300019.XSHE', '688218.XSHG', '688391.XSHG', '603803.XSHG', '688662.XSHG', '003013.XSHE', '002730.XSHE', '603881.XSHG', '300860.XSHE', '300401.XSHE', '002297.XSHE', '300419.XSHE', '688049.XSHG', '001336.XSHE', '002707.XSHE', '603305.XSHG'

In [7]:
import os
strategy_base_dir = f'{cur_dir}/strategy_benefits'
strategy_files = os.listdir(path=strategy_base_dir)
cnt = 0
for strategy_file in strategy_files:
    if strategy_file.endswith('.csv'):
        strategy_file_path = os.path.join(strategy_base_dir, strategy_file)
        plot_strategy(strategy_file_path, stock_kline_date_dict)
        cnt = cnt + 1
        if cnt == 100:
            break

2024-07-15 22:32:57.864 | INFO     | __main__:calculate_strategy:35 - first_buy_date_index: 484, first_buy_date 20240102


2024-07-15 22:32:57.866 | INFO     | __main__:calculate_strategy:64 - len holding_stock_list: 133, stock_code_set {'002921.XSHE', '301098.XSHE', '688511.XSHG', '300321.XSHE', '300865.XSHE', '003004.XSHE', '688350.XSHG', '301359.XSHE', '603272.XSHG', '688115.XSHG', '300960.XSHE', '688026.XSHG', '688171.XSHG', '688021.XSHG', '301237.XSHE', '688655.XSHG', '300906.XSHE', '688217.XSHG', '301079.XSHE', '300886.XSHE', '688656.XSHG', '301156.XSHE', '301231.XSHE', '301273.XSHE', '301209.XSHE', '688367.XSHG', '301075.XSHE', '300930.XSHE', '688227.XSHG', '301022.XSHE', '688466.XSHG', '301178.XSHE', '002802.XSHE', '688178.XSHG', '603580.XSHG', '300876.XSHE', '301105.XSHE', '688309.XSHG', '688379.XSHG', '688067.XSHG', '603159.XSHG', '300897.XSHE', '301192.XSHE', '301008.XSHE', '688395.XSHG', '301036.XSHE', '688218.XSHG', '688335.XSHG', '688455.XSHG', '688163.XSHG', '003008.XSHE', '301057.XSHE', '603090.XSHG', '688681.XSHG', '300838.XSHE', '688501.XSHG', '301233.XSHE', '300955.XSHE', '688355.XSHG', 

2024-07-15 22:32:57.941 | INFO     | __main__:calculate_strategy:35 - first_buy_date_index: 484, first_buy_date 20240102
2024-07-15 22:32:57.943 | INFO     | __main__:calculate_strategy:64 - len holding_stock_list: 2, stock_code_set {'688228.XSHG', '000501.XSHE'}


2024-07-15 22:32:57.987 | INFO     | __main__:calculate_strategy:35 - first_buy_date_index: 489, first_buy_date 20240109
2024-07-15 22:32:57.988 | INFO     | __main__:calculate_strategy:64 - len holding_stock_list: 3, stock_code_set {'600266.XSHG', '300005.XSHE', '300457.XSHE'}


2024-07-15 22:32:58.046 | INFO     | __main__:calculate_strategy:35 - first_buy_date_index: 485, first_buy_date 20240103
2024-07-15 22:32:58.047 | INFO     | __main__:calculate_strategy:64 - len holding_stock_list: 2, stock_code_set {'600455.XSHG', '300823.XSHE'}


...

2024-07-15 22:33:04.612 | INFO     | __main__:calculate_strategy:35 - first_buy_date_index: 508, first_buy_date 20240205


2024-07-15 22:33:04.670 | INFO     | __main__:calculate_strategy:35 - first_buy_date_index: 484, first_buy_date 20240102
2024-07-15 22:33:04.671 | INFO     | __main__:calculate_strategy:64 - len holding_stock_list: 0, stock_code_set set()


2024-07-15 22:33:04.731 | INFO     | __main__:calculate_strategy:35 - first_buy_date_index: 498, first_buy_date 20240122
2024-07-15 22:33:04.732 | INFO     | __main__:calculate_strategy:64 - len holding_stock_list: 1, stock_code_set {'300549.XSHE'}


2024-07-15 22:33:04.792 | INFO     | __main__:calculate_strategy:35 - first_buy_date_index: 490, first_buy_date 20240110
2024-07-15 22:33:04.793 | INFO     | __main__:calculate_strategy:64 - len holding_stock_list: 2, stock_code_set {'688057.XSHG', '688455.XSHG'}


2024-07-15 22:33:04.869 | INFO     | __main__:calculate_strategy:35 - first_buy_date_index: 496, first_buy_date 20240118
2024-07-15 22:33:04.870 | INFO     | __main__:calculate_strategy:64 - len holding_stock_list: 7, stock_code_set {'688296.XSHG', '301125.XSHE', '300594.XSHE', '688217.XSHG', '688193.XSHG', '688309.XSHG', '688021.XSHG'}
